In [2]:
import numpy as np
from math import ceil
# Run the fitness 
from oct2py import Oct2Py
from multiprocessing.pool import ThreadPool 
# Run multiple Zs as different processes
from multiprocessing import Pool
# to save the progress of the algorithms
import csv
from datetime import datetime
from os import mkdir

In [3]:
# -------- range of the variables ----------
V_S = 7.0                   # service speed [kn]
range_D     = [0.5, 0.8]
range_AEdAO = [0.3, 1.05]
range_PdD   = [0.5, 1.4]
range_Z     = [2, 7]

# Define the lower and upper bounds for each variable
lower_bounds = [range_D[0], range_AEdAO[0], range_PdD[0]]
upper_bounds = [range_D[1], range_AEdAO[1], range_PdD[1]]

In [4]:
# size of step from each variable
LIST_SIZE = {'D':     int((range_D[1]-range_D[0])        / 0.04),
             'AEdAO': int((range_AEdAO[1]-range_AEdAO[0])/ 0.01),
             'PdD':   int((range_PdD[1]-range_PdD[0])    / 0.02)}
LIST_SIZE

{'D': 7, 'AEdAO': 75, 'PdD': 44}

In [5]:
t = [LIST_SIZE[v] for v in LIST_SIZE]
all_iter = t[0]*t[1]*t[2]
all_iter

23100

In [15]:
150/30



5.0

In [17]:
# number of octave evaluations to run in parallel in each process
NUM_PARALLEL = 3

NUM_BATCHES = ceil(LIST_SIZE['PdD'] / NUM_PARALLEL)

3

In [48]:
# save the results, otherwise there is no output
save_file = True

print_octave_result = False

In [7]:
def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
    
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    append_to_file(filename, row)
    
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]", 
                  "AEdAO = expanded area ratio", 
                  "PdD = pitch ratio", 
                  "Z = propeller's number of blades", 
                  "P_B = power brake", 
                  "n = Propeller angular speed [rpm]", 
                  "fitness", 
                  "i = thread pool id"]
        writer.writerow(header) 
    return filename

def create_dir(text):
    now = datetime.now()
    dir_name = './' + text +'_'+ now.strftime("%Y_%m_%d_%H_%M")
    try:
        mkdir(dir_name)
    except: pass
    return dir_name

In [19]:
def run_octave_evaluation(V_S,D,Z,AEdAO,PdD):
    P_B, n = [0, 0]
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        P_B, n = octave.F_LabH2_aprox(V_S,D,Z,AEdAO,PdD, nout=2)
    return [P_B, n]

Logic for the iterations

In [40]:
def wrapper(k, x):
    D, AEdAO, PdD = x
    P_B, n = run_octave_evaluation(V_S,D,Z,AEdAO,PdD)
    return [k, P_B]

def run_list_parallel(x_list):
    fitness_list = np.zeros(len(x_list))
    # run eval in parallel
    with ThreadPool(len(x_list)) as pool:
        id_x = [(k, x_list[k]) for k in range(len(x_list))]
        for result in pool.starmap(wrapper, id_x):
            k, fitness = result
            fitness_list[k] = fitness
    # go through the results and save
    for k in range(len(x_list)):
        P_B = fitness_list[k]
        D, AEdAO, PdD = x_list[k]
        if print_octave_result:
            print("D:",D,"Z:",Z,"AEdAO:",AEdAO,"PdD:",PdD, "P_B:",P_B)
        if save_file:
            append_to_file(filename, [D, AEdAO, PdD, Z, P_B])

In [41]:
list_D     = np.linspace(range_D[0],     range_D[1],     LIST_SIZE['D'])
list_AEdAO = np.linspace(range_AEdAO[0], range_AEdAO[1], LIST_SIZE['AEdAO'])
list_PdD   = np.linspace(range_PdD[0],   range_PdD[1],   LIST_SIZE['PdD'])

In [43]:
def run(z):
    global Z
    Z = z
    # create the csv file with the headers
    if save_file:
        global filename
        filename = create_file(str(Z))

    counter = 0
    for D in list_D:
        for AEdAO in list_AEdAO:
            # divide PdD in batches of size NUM_PARALLEL
            for b in range(NUM_BATCHES):
                start = b * NUM_PARALLEL
                end   = (b+1) * NUM_PARALLEL
                list_x = [(D,AEdAO,PdD) for PdD in list_PdD[start:end]]
                # run list in parallel
                run_list_parallel(list_x)
                # keep count of the progress
                counter += len(list_x)
                print('Z:',Z, counter,'/',all_iter)
                    
    return Z

parallel

In [49]:
# create dir named 'gradient'
global dir_name
dir_name = create_dir('gradient')
    
if __name__ == '__main__':
    with Pool() as pool:
        for result_z in pool.map(run, range(range_Z[0],range_Z[1]+1)):
            print('end of ', result_z)

D:D:  0.50.5 Z:  5 Z: 4 AEdAO: 0.3 PdD: AEdAO: 0.50.3 PdD: 0.5  P_B:D:P_B: 0.0 0.0
 D:0.5 
 D:0.5Z: 0.5 Z:  3 5 AEdAO:Z:   40.3 PdD:  AEdAO:0.5  0.3AEdAO: P_B: PdD: 0.6285714285714286  0.0
0.3D:P_B: D:  0.0 
0.5PdD: 0.5  0.6285714285714286Z:D:  3 Z: 0.5 Z:P_B:AEdAO:  4 2  0.3 0.0AEdAO: AEdAO:
  D:PdD:0.30.3  0.6285714285714286  P_B:PdD:   0.50.00.5 PdD:P_B: 
Z:  0.0D: 5 0.7571428571428571 
AEdAO:  P_B:0.3 D:0.00.5
   Z:PdD:0.5  4 0.7571428571428571Z:   32Z:  AEdAO:P_B: 3 0.3  PdD: 0.6285714285714286/ 0.0P_B:  
AEdAO:0.0 
Z:D: 5 0.3 0.5   Z:336 2 PdD:AEdAO:3
   0.75714285714285710.3  /PdD:P_B:  0.0336 
0.7571428571428571Z:
  3P_B:  30.0
 Z: 2/ 3  /336 
336
D: 0.5 Z: 4 AEdAO: 0.3 PdD: 0.8857142857142857 D:P_B: 0.5 0.0
 D:Z:  50.5  Z: 4 AEdAO:AEdAO:  0.30.3 PdD:  PdD:0.8857142857142857  1.0142857142857142 P_B:P_B:  0.00.0
D:
 0.5D:  Z: 0.54  AEdAO:Z:  50.3  PdD:AEdAO: 0.3  PdD: 1.1428571428571428 P_B:1.0142857142857142  0.0P_B:
 Z: 0.0
4D:  0.5 6Z:  /5  336AEdAO:
 0.3 PdD: 1.1428571428571

 Z: PdD:PdD:3   0.7571428571428571 75P_B:1.1428571428571428   P_B:0.0/  
3360.0

Z: Z: 5 4 7884  //  336336

D: 0.56 Z: 4 AEdAO: 0.425 PdD: 1.2714285714285714 P_B: 0.0
D: 0.56 Z: 4 AEdAO: 0.425 PdD: 1.4 P_B: 0.0
Z: 4 81 / 336
D: 0.56 Z: 5 AEdAO: 0.55 PdD: 0.8857142857142857 P_B: 0.0
D: 0.56 Z: 5 AEdAO: 0.55 PdD:D:  1.0142857142857142 0.56P_B:  0.0
Z: D:2 0.56  AEdAO:Z:  50.425  AEdAO: PdD: 0.55 0.8857142857142857 PdD: P_B:1.1428571428571428  P_B:0.0 0.0
Z: 
5D: 0.56D: 87  Z: 0.56 / Z: 3336  
2 AEdAO:AEdAO: 0.425 0.425  PdD:PdD:  0.88571428571428571.0142857142857142  P_B:P_B: 0.0 
0.0
D:D: 0.56 Z:  2 0.56AEdAO: 0.425 Z: PdD:  1.14285714285714283 P_B: 0.0 
Z: AEdAO:2  0.42578 PdD: / 1.0142857142857142 336 
P_B: 0.0
D: 0.56 Z: 3 AEdAO: 0.425 PdD: 1.1428571428571428 P_B: 0.0
Z: 3 78 / 336
D: 0.56 Z: 5 AEdAO: 0.55 PdD: 1.2714285714285714 P_B: 0.0
D: 0.56 Z: 5 AEdAO: 0.55 PdD: 1.4 P_B: 0.0
Z: 5 90 / 336
D: 0.56 Z: 4 AEdAO: 0.55 PdD: 0.5 P_B: 0.0
D: D: 0.560.56  D:Z:Z:  2  0.56 4AEdAO:Z:   3A

D: 0.62 Z: 5 AEdAO: 0.8 PdD: 0.6285714285714286 P_B: 0.0
D: 0.62 Z: 5 AEdAO: 0.8 PdD: 0.7571428571428571 P_B: 0.0
Z: 5 165 / 336
D: 0.62 Z: 3 AEdAO: 0.55 PdD: 1.2714285714285714 P_B: 0.0
D: 0.62 Z: 3 AEdAO: D: 0.55 PdD:0.62  1.4Z:  P_B:4 0.0 
AEdAO:Z:  0.6753  PdD:153  0.5 P_B:/ 0.0 
D:336 
0.62 Z:D:  0.624 Z:  AEdAO:2  0.675AEdAO:  0.55PdD:  0.6285714285714286PdD:  P_B: 1.27142857142857140.0 P_B:
D:  0.620.0 
D:Z:  0.624 Z: AEdAO:  0.6752  AEdAO:PdD:  0.550.7571428571428571  PdD:P_B:  0.01.4 
P_B: 0.0Z:
 4Z: 2  153156  //  336336

D: 0.62 Z: 5 AEdAO: 0.8 PdD: 0.8857142857142857 P_B: 0.0
D: 0.62 Z: 5 AEdAO: 0.8 PdD: 1.0142857142857142 P_B: D:0.0 0.62 Z: 
D:4 0.62 Z: 5 AEdAO: AEdAO: 0.8  PdD: 1.1428571428571428 0.675P_B:  PdD:0.0
 Z: 0.88571428571428575  168P_B:  0.0/
 336
D: 0.62 Z: 4 AEdAO: 0.675 PdD: D:1.0142857142857142 0.62  Z:P_B:  0.03
 AEdAO:D:  0.62 Z:0.675 PdD: 4 AEdAO:  0.50.675  PdD:P_B: 0.0 1.1428571428571428
 D:P_B: 0.62 0.0
 Z:Z:  34  159AEdAO: 0.675  / PdD: 0.62857142857

D: 0.68 Z: 4 AEdAO: 0.8 PdD: 1.1428571428571428 P_B: 0.0
Z: 4 231 / 336
D: 0.68 Z: 5 AEdAO: 0.925 PdD: 1.2714285714285714 P_B: 0.0
D: 0.68 Z: 5 AEdAO: 0.925 PdD: 1.4 P_B: 0.0
Z: 5 243 / 336
D: 0.68 Z: 3 AEdAO: 0.8 PdD: 0.5 P_B: 0.0
D: 0.68 Z: 3 AEdAO: 0.8 PdD: 0.6285714285714286 P_B: 100.27730212614887
D: 0.68 Z: 3 AEdAO: 0.8 PdD: 0.7571428571428571 P_B: 0.0
Z: 3D:  228 0.68/  Z: 3362
 AEdAO: 0.8 PdD: 0.5 D:P_B:  0.68 0.0Z:
 D:4  0.68AEdAO: 0.8  Z:PdD:  2 1.2714285714285714AEdAO:  0.8P_B:  PdD:0.0
 D:0.6285714285714286 P_B: 0.68 107.06848425054216
 D: Z: 0.684  Z: AEdAO:2  0.8AEdAO:  PdD:0.8  PdD:1.4  0.7571428571428571 P_B:P_B:  0.0
0.0Z: 4 234 
Z:/ 2  336228 
/ 336
D: 0.68 Z: 5 AEdAO: 1.05 PdD: 0.5 P_B: 0.0
D: 0.68 Z: 5 AEdAO: 1.05 PdD: 0.6285714285714286 P_B: 98.33870533242793
D: 0.68 Z: 5 AEdAO: 1.05 PdD: 0.7571428571428571 P_B: 96.81280473801294
Z: 5 246 / 336
D:D:  0.680.68 Z:  3Z:  AEdAO:  20.8 AEdAO:PdD:  0.80.8857142857142857 P_B:  0.0PdD:
 0.8857142857142857 D:P_B: 0.0
 D:0.6

D: 0.74 Z: 4 AEdAO: 0.925 PdD: 1.2714285714285714 P_B: 0.0
D: 0.74 Z: 4 AEdAO: 0.925 PdD: 1.4 P_B: 0.0
Z: 4 306 D:/  0.8 Z:336 5 
AEdAO: 0.3 PdD: 0.5 P_B: 0.0
D: 0.8 Z: 5 AEdAO: 0.3 PdD: 0.6285714285714286 P_B: 0.0
D: 0.8 Z: 5 AEdAO: 0.3 PdD:D: 0.7571428571428571 0.74 P_B:  0.0Z: 2 
AEdAO:Z: 0.925 5 318  /PdD:  336
0.8857142857142857 P_B: 100.45497931061252D:
 D:0.74  Z:0.74 Z: 3 2  AEdAO: AEdAO:0.925 PdD:  0.5 P_B:0.925  0.0
PdD:D:  0.741.0142857142857142  Z:P_B:  0.03
D: AEdAO:  0.9250.74  Z:PdD: 0.6285714285714286 P_B: 96.71161190060342 2 AEdAO:
 D: 0.9250.74 PdD: Z:  31.1428571428571428  P_B: AEdAO: 0.00.925
 PdD:Z:  20.7571428571428571  303P_B: 94.65031119323852 
/Z: 336 3
 300 / 336
D: 0.8 Z: 5 AEdAO: 0.3 PdD: 0.8857142857142857 P_B: 0.0
D: 0.8 Z: 5 AEdAO: 0.3 PdD: 1.0142857142857142 P_B: 0.0
D: 0.8 Z: 5 AEdAO: 0.3 PdD: D:1.1428571428571428  0.74P_B:  Z: 4 AEdAO: 1.05 0.0PdD:
Z: 5 321 / 336
 0.5 P_B: 0.0
D: 0.74 Z: 4 AEdAO: 1.05 PdD: 0.6285714285714286 P_B: 94.88060215094225
D: 0

D:PdD:  1.01428571428571420.8 P_B:  Z:88.5603718018849 
D: 3 0.8 Z:AEdAO:  0.9254  PdD: 0.8857142857142857AEdAO: 1.05  PdD:P_B: 1.1428571428571428 89.79084886586172
 P_B:D:  0.891.19109617465877 
Z:Z:  43  AEdAO:375 0.925 PdD:  / 1.0142857142857142336 P_B:
 91.7271843237487
D: 0.8 Z: 3 AEdAO: 0.925 PdD: 1.1428571428571428 P_B: 0.0
Z: 3 366 / 336
D: 0.8 Z: 2 AEdAO: 1.05 PdD: 0.8857142857142857 P_B: 92.59754320455363
D: 0.8 Z: 2 AEdAO: 1.05 PdD: 1.0142857142857142 P_B: 91.81128036872357
D: 0.8 Z: 2 AEdAO: 1.05 PdD: 1.1428571428571428 P_B: 91.15084508375766
Z: 2 375 / 336
D: 0.5 Z: 6 AEdAO: 0.55 PdD:D:  0.8857142857142857 0.8 P_B:Z:  0.0
4 D:AEdAO:  1.050.5  PdD:Z:  6 1.2714285714285714AEdAO:  0.55 PdD: 1.0142857142857142 P_B: 0.0
D:P_B:  0.50.0 Z:
 6D:  AEdAO:0.8  0.55D:Z: 0.8  4  PdD:AEdAO:  Z:1.051.1428571428571428  PdD:3  1.4P_B:   AEdAO: 0.0P_B:
0.925Z:  60.0
 Z: PdD:24  1.27142857142857144   / P_B: 3780.0336 /

 D:336 0.8 Z: 3
 AEdAO: 0.925 PdD: 1.4 P_B: 0.0
Z: 3 369 / 336
D: 0.5 D:

Z:  6 AEdAO:7 0.675  147PdD:  0.7571428571428571 P_B: /0.0
 336
Z: 6 156 / 336
D: 0.62 Z: 7 AEdAO: 0.55 PdD: 0.8857142857142857 P_B: 0.0
D: 0.62 Z: 7 AEdAO: 0.55 PdD: 1.0142857142857142 P_B: 0.0
D: 0.62 Z: 7 AEdAO: 0.55 PdD: 1.1428571428571428 P_B: 0.0
Z: 7 150 / 336
D: 0.62 Z: 7 AEdAO: 0.55 PdD: 1.2714285714285714 P_B: 0.0
D:D:  0.62 0.62 Z: Z:6 AEdAO: 0.675 7  AEdAO:PdD: 0.8857142857142857 P_B: 0.0
 D:0.55  0.62PdD:  1.4Z:  6 AEdAO: 0.675 PdD: 1.0142857142857142 P_B: P_B:0.0 0.0

Z: 7 153 D:/ 336 
0.62 Z: 6 AEdAO: 0.675 PdD: 1.1428571428571428 P_B: 0.0
Z: 6 159 / 336
D: 0.62 Z: 7 AEdAO: 0.675 PdD: 0.5 P_B: 0.0
D: 0.62 Z: 7 AEdAO: 0.675 PdD: 0.6285714285714286 P_B: 0.0
D: 0.62 Z: 7 AEdAO: 0.675 PdD: 0.7571428571428571 P_B: 0.0
Z: 7 156 / 336
D: 0.62 Z: 6 AEdAO: 0.675 PdD: 1.2714285714285714 P_B: 0.0
D: 0.62 Z: 6 AEdAO: 0.675 PdD: 1.4 P_B: 0.0
Z: 6 162 / 336
D: 0.62 Z: 7 AEdAO: 0.675 PdD: 0.8857142857142857 P_B: 0.0
D: 0.62 Z: 7 AEdAO: 0.675 PdD: 1.0142857142857142 P_B: 0.0
D: 0.62 Z: 

D: 0.74 Z: 6 AEdAO: 0.925 PdD: 1.0142857142857142 P_B: 0.0
D: 0.74 Z: 6 AEdAO: 0.925 PdD: 1.1428571428571428 P_B: 0.0
Z: 6 303 / 336
D: 0.74 Z: 7 AEdAO: 0.925 PdD: 0.8857142857142857 P_B: 90.17824522893606
D: 0.74 Z: 7 AEdAO: 0.925 PdD: 1.0142857142857142 P_B: 0.0
D: 0.74 Z: 7 AEdAO: 0.925 PdD: 1.1428571428571428 P_B: 0.0
Z: 7 303 / 336
D: 0.74 Z: 6 AEdAO: 0.925 PdD: 1.2714285714285714 P_B: 0.0
D: 0.74 Z: 6 AEdAO: 0.925 PdD: 1.4 P_B: 0.0
Z: 6 306 / 336
D: 0.74 Z: 7 AEdAO: 0.925 PdD: 1.2714285714285714 P_B: 0.0
D: 0.74 Z: 7 AEdAO: 0.925 PdD: 1.4 P_B: 0.0
Z: 7 306 / 336
D: 0.74 Z: 6 AEdAO: 1.05 PdD: 0.5 P_B: 0.0
D: 0.74 Z: 6 AEdAO: 1.05 PdD: 0.6285714285714286 P_B: 91.40232617546108
D: 0.74 Z: 6 AEdAO: 1.05 PdD: 0.7571428571428571 P_B: 90.29711756447423
Z: 6 309 / 336
D: 0.74 Z: 7 AEdAO: 1.05 PdD: 0.5 P_B: 0.0
D: 0.74 Z: 7 AEdAO: 1.05 PdD: 0.6285714285714286 P_B: 92.22651268679365
D: 0.74 Z: 7 AEdAO: 1.05 PdD: 0.7571428571428571 P_B: 91.52456612526687
Z: 7 309 / 336
D: 0.74 Z: 6 AEdAO: 1

results